<a href="https://colab.research.google.com/github/jtlagumbay/cebqa/blob/main/reader/cebqa_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CebQA Reader Component**
Pretrained model: RoBERTa

# **Libraries**

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import RobertaForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from huggingface_hub import login


ModuleNotFoundError: No module named 'datasets'

# **Loading Dataset**

In [ ]:
data = pd.read_csv('path/to/your-dataset.csv')

dataset = Dataset.from_pandas(data)

# **Tokenize Dataset**

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# **Dataset Splitting**

In [ ]:
train_data, val_data = train_test_split(tokenized_dataset, test_size=0.2)

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

# **Load Pre-Trained RoBERTa**

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)


# **Model Training**

## Training Argument

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

## Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()


## Saving the fine-tuned model

In [ ]:
model.save_pretrained("path/to/save-directory")
tokenizer.save_pretrained("path/to/save-directory")

# **Evaluating the model**

## Evaluating

In [ ]:
results = trainer.evaluate()
print(results)

## Inference

In [ ]:
inputs = tokenizer("Your input text here", return_tensors="pt", truncation=True, padding="max_length")
outputs = model(**inputs)
logits = outputs.logits
predicted_class = logits.argmax().item()
print(f"Predicted class: {predicted_class}")